In [ ]:
# clean output directory every time
import shutil
try: shutil.rmtree('out')
except FileNotFoundError: pass

from pathlib import Path
Path('out').mkdir(parents=True, exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

In [ ]:
# data files are exactly as received from MIRA Helsinki Study. This code does not write to these files.
data_master = 'data/mira/MIRA Master file 180527.sav'
data_lab = 'data/mira/HUSLAB Data final_WithMixedVegans.txt'
data_intakes = 'data/mira/muuttujat_analyysiin.txt'
data_thl = 'data/mira/growth-curves.tsv'
data_questionnaire = 'data/mira/data huoltajan tausta 171106.sav'
data_food_record = 'data/mira/radata.txt'

In [ ]:
# global settings for graph output
scale=10
sns.set_theme(style='white',font_scale=2)

In [ ]:
#declare common column names as variables
ldl = 'fP-Kol-LDL (mmol/l)'
tc = 'fP-Kol (mmol/l)'
serum_lipids = [
    tc,
    ldl, 
    'fP-Kol-HDL (mmol/l)', 
    'fP-Trigly (mmol/l)'
]

In [ ]:
# read in mira study data
lab_results = pd.read_csv(data_lab, sep='\t', decimal=",")
intakes = pd.read_csv(data_intakes, sep='\t')

subjects_all = intakes.merge(lab_results, how='left', on='ID')

#df = lab_results.merge(intakes, on='ID')
# Only select subjects for whom we have an LDL lab result 
#subjects = df[df[ldl].notna()]
#subjects = df

In [ ]:
# classify diets
g6_map = {
    'Pesco-vegetarian': 'Vegetarian',
    'Vegan': 'Vegan',
    'Control': 'Omnivore',
    'Control (vegan in daycare)': 'Omnivore',
    'Vegetarian': 'Vegetarian',
}
diet_class = 'diet classification'
diet_classes = ['Vegan','Vegetarian','Omnivore']

s = subjects_all.Group4.map(g6_map)
s = s.fillna('Omnivore')

subjects_all.insert(6,diet_class,s)

In [ ]:
subjects_all[['ID','Group2','Group3','Group4',diet_class]].to_csv('out/subject_grouping.csv')

In [ ]:
# Use Finnish THL curves for child BMI SDS

curves = pd.read_csv(data_thl,sep='\t',decimal=',')
curves.columns = [c.lower() for c in curves.columns]

# no curve for children under two, fill in with a linear extrapolation

def fill_start_with_linear_extrapolation(s):
    i = s.first_valid_index()
    x1 = s.loc[i]
    x2 = s.loc[2*i]
    x0 = x1 - (x2-x1)
    return pd.Series(np.linspace(x0,x1,i)).append(s[i:])


for c in 'bmi_mean_m','bmi_sd_m','bmi_nu_m','bmi_mean_f','bmi_sd_f','bmi_nu_f':
    s = curves[c]
    s2 = fill_start_with_linear_extrapolation(s)
    curves.insert(
        curves.columns.get_loc(c)+1,
        c+'_filled',
        s2
    )

# BMISDS = ((BMIlaskettu / muBMI) ^ nuBMI – 1) / (nuBMI × sigmaBMI)
def translate_sex(s):
    if s in ('M','m'): return 'm'
    if s in ('N','n','F','f'): return 'f'
    return None

def bmi_sds(weight,height,age,sex):
    # NaN check
    if age != age: return None
    
    age = round(age,2)
    sex = translate_sex(sex)

    bmi = weight/height**2

    row = curves[curves.age==age].iloc[0]
    mu_bmi = row['bmi_mean_'+sex+'_filled']
    nu_bmi = row['bmi_nu_'+sex+'_filled']
    sigma_bmi = row['bmi_sd_'+sex+'_filled']

    bmi_sds = ((bmi/mu_bmi)**nu_bmi - 1) / (nu_bmi * sigma_bmi)

    return bmi_sds

bmi_sds = subjects_all.apply(
    lambda row: bmi_sds(
        row.Weight,
        row.Height/100,
        row.AntAge, #row.Bage,
        row.Sex
    ),
    axis=1
)
subjects_all.insert(12,'bmi_sds',bmi_sds)

In [ ]:
# classify BMI by SDS

def is_female(s):
    return s in ('N','n','F','f')

def is_male(s):
    return s in ('M','m')

def classify_bmi_sds(bmi_sds,sex):
    if is_female(sex):
        if bmi_sds < -2.2187: return 'Significantly underweight'
        if bmi_sds < -1.6482: return 'Underweight'
        if bmi_sds > 2.7600: return 'Severely Obese'
        if bmi_sds > 2.1065: return 'Obese'
        if bmi_sds > 1.1629: return 'Overweight'
        return 'Normal'
    if is_male(sex):
        if bmi_sds < -2.3456: return 'Significantly underweight'
        if bmi_sds < -1.8344: return 'Underweight'
        if bmi_sds > 2.3600: return 'Severely Obese'
        if bmi_sds > 1.7016: return 'Obese'
        if bmi_sds > 0.7784: return 'Overweight'
        return 'Normal'
    return None

bmi_class = subjects_all.apply(
    lambda row: classify_bmi_sds(
        row.bmi_sds,
        row.Sex
    ),
    axis=1
)

subjects_all.insert(13,'bmi_class',bmi_class)

In [ ]:
subjects_all['ENER_kcal_per_d'] = subjects_all.ENERJ_per_d / 4.184

In [ ]:
# animal proportion per food item collected for thesis, method and sources on page 39 of thesis
data_food_animal_proportion = 'data/food-animal-proportion.csv'

In [ ]:
food_records = pd.read_csv(
    data_food_record, 
    sep='\t', 
    encoding='iso-8859-1')

food_animal_percent = pd.read_csv(
    data_food_animal_proportion, 
    index_col='code'
).drop(columns=['link'])

food_records = food_records.merge(
    food_animal_percent, 
    left_on='Code', 
    right_on='code', 
    how='left'
)
food_records['timestamp'] = pd.to_datetime(
    food_records.DaDate + ' ' + food_records.MaTime,
    format='%d.%m.%Y %H:%M:%S'
)

In [ ]:
#calculate age when food records taken
master = pd.read_spss(data_master)

df = pd.DataFrame()
df['dob'] = pd.to_datetime(
    master.set_index('ID').syntymaaika
)
df['date_of_first_food_record'] = pd.to_datetime(
    food_records.groupby('ID').DaDate.min()
)
df['age_at_first_food_record'] = df.date_of_first_food_record - df.dob

subjects_all = subjects_all.merge(
    df.drop(columns=['dob']).reset_index()
)

In [ ]:
#sanity checking food record lenths
df = food_records.groupby('ID').agg({'timestamp': ['min', 'max']})
df.columns = ['ts_min','ts_max']
df = df.merge(
    food_records[['ID','DaDate']].groupby('ID').nunique(),
    on='ID'
)
df.rename(columns = {'DaDate':'record_distinct_dates'}, inplace = True)
df.record_distinct_dates.hist(range=(0,6),bins=6)

In [ ]:
# sanity check which food items with nonzero animal content appear for vegans

df = food_records.merge(
    subjects_all[['ID','Group4']], 
    on='ID', 
    how='left')
df = df[
    (df.animal_proportion > 0) 
    & (df.Group4 == 'Vegan')
]
df.groupby(['name','MaName']).count()['Group4']

In [ ]:
# calculate ASE proportion
r_animal = 'r_animal_source_energy'
r_animal_label = 'Animal source energy proportion'

e = food_records[['ID','ENERJ','animal_proportion']]
e = e.assign(ENERJ_animal=(e.ENERJ * e.animal_proportion)).drop(columns=['animal_proportion'])

#energy per subject
eps = e.groupby(['ID']).sum()
eps = eps.assign(r_animal_source_energy=(eps.ENERJ_animal / eps.ENERJ))

subjects_all = subjects_all.merge(eps, on='ID')

# Subject selection

In [ ]:
# data has been manually verified
# TODO: implement subject selection in code here
fr_too_short = (105, 405, 801, 802, 119)
fr_no_weekend = (404,)
fr_invalid = fr_too_short + fr_no_weekend

subjects_fr = subjects_all[~subjects_all.ID.isin(fr_invalid)]
subjects_ldl = subjects_fr[subjects_fr[ldl].notna()]

## Figure 1

Distribution of the ASE proportions by the diet classification (grouping based on the food records and background questionnaires). Participants' dietary classification is indicated with color coding, with blue for vegan, orange for vegetarian and green for omnivore group. The vegetarian group included lactovegetarians, lacto-ovo-vegetarians and pescovegetarians.

In [ ]:
sns.set_context("paper")

tricolor_palette=['#55a868','#dd8452','#4c72b0']

plt.close()
fg = sns.displot(
    subjects_fr,
    x=r_animal,
    binwidth=0.05,
    hue=diet_class,
    hue_order=diet_classes,
    multiple='stack',
    height=scale,
    palette=tricolor_palette
)
fg.axes[0,0].set_xlabel(r_animal_label)
fg.axes[0,0].set_ylabel('Number of participants')

plt.savefig('out/Fig1.svg')
plt.savefig('out/Fig1.png')

# Table 1
Describing data with median (min-max).

In [ ]:
intakes = [
    'ENERJ_per_d',
    'ENER_kcal_per_d',
    'FAT_e_pros',
    'FASAT_e_pros',
    'FAMS_e_pros',
    'FAPU_e_pros',
    'CHOL_per_MJ',
    'CHOL_per_d',
    'PROT_e_pros',
    'CHO_e_pros',
    'SUCS_e_pros',
    'FIBC_per_MJ',
    'FIBC_per_d',
    'SALT_per_MJ',
    'SALT_per_d',
    'FOL_per_d'
]
biomarkers = [
    'fP-Kol (mmol/l)',
    'fP-Kol-LDL (mmol/l)',
    'fP-Kol-HDL (mmol/l)',
    'fP-Trigly (mmol/l)',
    'fE-Folaat (nmol/l)'    
]

In [ ]:
def summarize(df):
    print(len(df.index))
    display(df.sukupuoli.value_counts().to_frame())
    display(df.bmi_class.value_counts().to_frame())
    display(
        df[['age_at_first_food_record','bmi_sds']+intakes+biomarkers].describe().transpose()[['50%','min','max']]
    )

In [ ]:
summarize(subjects_fr)

In [ ]:
summarize(subjects_ldl)

# Table 2

In [ ]:
def correlation_values(subjects, xs,ys):
    table = []
    for x in xs:
        for y in ys:
            df = subjects[[x,y]].dropna()
            pr, pp = pearsonr(df[x],df[y])
            table.append([x,y,pr,pp])

    df = pd.DataFrame(table,columns=['x','y','pearson_r','pearson_p'])
    df['fdr_bh_0_05'] = multipletests(df['pearson_p'], alpha=0.05, method='fdr_bh')[0]
    df = df.sort_values(by='pearson_p')
    df = df.round(3)
    return df

In [ ]:
df = correlation_values(
    subjects_ldl,
    [r_animal],
    intakes
)
df.to_csv('out/table2-correlations-ASEP-intakes.csv')
df

# Figure 2

In [ ]:
from string import ascii_lowercase

markers=['o','s','^']

def regplot_r_animal(df,ax,y,y_label,set_xlabel=True):
    ax.set_xlim(-0.01, df.r_animal_source_energy.max()+0.01)
    sns.regplot(ax=ax, x=r_animal, y=y, scatter=False, data=df)
    sns.scatterplot(
        ax=ax, 
        x=r_animal, 
        y=y, 
        hue=diet_class,
        hue_order=diet_classes,
        style=diet_class,
        markers=markers,
        data=df, 
        s=30*scale,
        palette=tricolor_palette,
        legend=False
    )
    if set_xlabel:
        ax.set_xlabel(r_animal_label)
    else:
        ax.set_xlabel(None)
    ax.set_ylabel(y_label)


from matplotlib.lines import Line2D
def splats(n):
    return [
        Line2D(
            [0], 
            [0], 
            marker=markers[i],
            color='w',
            markerfacecolor=tricolor_palette[i],
            markersize=15
        )
        for i in range(0,n)
    ]


def fig_of_regplots(df,ys,cols=2):

    plt.close()
    sns.set_theme(style='white',font_scale=2)
    
    rows = int(len(ys)/cols)+(len(ys)%cols > 0)
    h = rows*scale
    if rows > 1: h+=2
    w = cols*scale+2
    


    fig, axs = plt.subplots(rows,cols,figsize=(w,h))
        
    if rows > 1:
        axs_flat = [ax for row in axs for ax in row]
    else:
        axs_flat = axs

    l = list(zip(ys.keys(),ys.values(),axs_flat))
    i = 0
    for t in l:
        ax = t[2]
        y = t[0]
        regplot_r_animal(df,ax,y,t[1],set_xlabel=False)
        i+=1

    for ax in axs_flat[len(ys):]:
        fig.delaxes(ax)
        
    from matplotlib.lines import Line2D
    
    plt.figlegend(splats(3),diet_classes,loc='right')
        
    return fig, axs

In [ ]:
sns.set_context("paper")

ys = {
    'PROT_e_pros': 'Protein (E%)',
    'FIBC_per_MJ': 'Fiber (g/MJ)',
    'FOL_per_d': 'Folate (µg/d)',
    'FASAT_e_pros': 'Saturated Fat (E%)',
    'FAPU_e_pros': 'Polyunsaturated Fat (E%)',
    'CHOL_per_MJ': 'Cholesterol (mg/MJ)',
}

fig, axs = fig_of_regplots(subjects_ldl,ys,cols=3)

plt.savefig('out/Fig2.A.svg')
plt.savefig('out/Fig2.A.png')

plt.show()

In [ ]:
sns.set_context("paper")

ys = {
    'fP-Kol-LDL (mmol/l)': 'LDL cholesterol (mmol/l)',
    'fE-Folaat (nmol/l)': 'Erythrocyte folate (nmol/l)'  
}


fig, axs = fig_of_regplots(subjects_ldl,ys,cols=3)


plt.savefig('out/Fig2.B.svg')
plt.savefig('out/Fig2.B.png')

plt.show()

# Exploration

In [ ]:
#median ASEP for non Vegans
subjects_fr[subjects_fr['diet classification'].isin(['Vegetarian','Omnivore'])].r_animal_source_energy.median()

In [ ]:
sns.set_context("paper")

ys = {
    'B-Hb (g/l)': 'Hemoglobin (g/l)',
    'S-Ferritin': 'Serum ferritin ',
}
df = subjects_fr
for k in ys.keys():
    df = df[df[k].notna()]

fig, axs = fig_of_regplots(df,ys,cols=2)

plt.show()

In [ ]:
correlation_values(
    subjects_fr,
    [r_animal],
    ['B-Hb (g/l)','S-Ferritin']
)